In [ ]:
!pip install pydub # imported in classification_utils

import os
import h5py
import random
import csv

import keras
from keras.optimizers import Adam
from keras.applications import VGG19
from keras.utils import to_categorical, plot_model
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation, Flatten, Dropout, Input
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/')

In [ ]:
# Auto re-load utils from util py files
%load_ext autoreload
%autoreload 2

In [ ]:
%cd "path/to/working/folder"
!ls

from classification_utils import *

# Read the dataset

In [ ]:
hdf5_file = 'file-to-save-img-samples.hdf5'

f =  h5py.File(hdf5_file, "r+")
specs_h5 = np.array(f["specs"]).astype("float32")
labels_h5 = np.array(f["labels"])
sample_source_h5 = np.array(f["sample_source"])
f.close()

In [ ]:
# Set random seed to preserve same train, test, and val in case codes were
# terminated due to restrained resources 
random.seed(100)

typeUsed= np.unique(labels_h5)
print(typeUsed)

fixed_num = 134
upsample_data = False

log_scale = False
print("Using log scale", log_scale)
if log_scale:
  specs_h5  = np.where(specs_h5 > 1.0e-10, specs_h5, 1.0e-10)
  specs_h5 = 10*np.log10(specs_h5)

specs = np.empty((0,224, 224, 3))
labels = np.empty((0,1))
x_test = np.empty((0,224, 224, 3))
y_test = np.empty((0,1))
x_val = np.empty((0,224, 224, 3))
y_val = np.empty((0,1))

for i in range(len(typeUsed)):
  # get index of the current type of spec
  cur_index = np.argwhere(labels_h5 == typeUsed[i]).flatten()
  min_num = min(len(cur_index), fixed_num)

  if typeUsed[i] == b'noise-or-background':
    # Ensure new noises of winds are included.
    cur_index = cur_index[-min_num:]
  
  print(typeUsed[i], len(cur_index), min_num)
  # to ensure we have same number of type b and i
  random.shuffle(cur_index)
  cur_index_resized = cur_index[:int(min_num * 0.8)]
  test_index = cur_index[int(min_num * 0.8): int(min_num * 0.9)]
  val_index = cur_index[int(min_num * 0.9): min_num]

  cur_x_train = specs_h5[cur_index_resized]
  cur_x_val = specs_h5[val_index]
  cur_x_test = specs_h5[test_index]
  
  if upsample_data:
    target_x_train_num = int(fixed_num * 0.8)
    target_x_val_num = int(fixed_num * 0.1)
    target_x_test_num = int(fixed_num - target_x_train_num - target_x_val_num)
    cur_x_train = upsample(cur_x_train, target_x_train_num)
    cur_x_val = upsample(cur_x_val, target_x_val_num)
    cur_x_test = upsample(cur_x_test, target_x_test_num) 

  specs = np.append(specs, cur_x_train, axis = 0)
  x_test = np.append(x_test, cur_x_test, axis = 0)
  x_val = np.append(x_val, cur_x_val, axis = 0)
  labels = np.append(labels, np.repeat(i, len(cur_x_train)))
  y_val = np.append(y_val, np.repeat(i, len(cur_x_val)))
  y_test = np.append(y_test, np.repeat(i, len(cur_x_test)))

# specs_keep = np.copy(specs)

In [ ]:
# Normalize
specs = np.array(normalize(specs))
x_test = np.array(normalize(x_test))
x_val = np.array(normalize(x_val))
specs.shape, x_test.shape, x_val.shape

In [ ]:
# Categorize labels
cat_labels = to_categorical(pd.factorize(labels)[0],num_classes= len(typeUsed))
cat_y_test = to_categorical(pd.factorize(y_test)[0],num_classes= len(typeUsed))
cat_y_val = to_categorical(pd.factorize(y_val)[0],num_classes= len(typeUsed))

print("test size:", len(y_test))
print("train size", len(labels))
print("val size", len(y_val))

# Train

## Model

In [ ]:
def build_finetune_model(base_model, dropouts, fc_layers, num_classes):
    # transfer learning, freeze model
    for layer in base_model.layers:
       layer.trainable = False

    x = base_model.output
    x = Flatten()(x)

    for fc, drop in zip(fc_layers, dropouts):
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(drop)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [ ]:
class TestCallback(keras.callbacks.Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

## Train

In [ ]:
config = dict(
    dropout = 0.5,
    hidden = 32,
    learn_rate = 1e-5,
    batch_size= 32,
    epoch = 400
    )

prefix = "model_file_prefix"

In [ ]:
model = None
keras.backend.clear_session()

filepath_loss = prefix + 'model_loss.hdf5'
filepath_acc = prefix + 'model_acc.hdf5'

shape_x = 224
shape_y = 224
model = VGG19(weights='imagenet', include_top=False, input_shape=(shape_x,shape_y,3))
model = build_finetune_model(model, 
                            [config["dropout"], config["dropout"]], 
                            [config["hidden"], config["hidden"]], 
                            len(typeUsed))

# Print and plot model architecture
model.summary()
plot_model(model,to_file = 'path/to/plot.png')

print("Using learning rate:", config["learn_rate"])
opt = Adam(learning_rate=config["learn_rate"])
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# remove model before
if os.path.exists(filepath_loss):
  os.remove(filepath_loss)
if os.path.exists(filepath_acc):
  os.remove(filepath_acc)

checkpoint_loss = ModelCheckpoint(filepath_loss, monitor='val_loss', mode='auto', verbose=1, save_best_only=True, save_weights_only=False, save_freq='epoch')
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

print("Training with batch size:", config["batch_size"])
# Not used: upsampling with data augmentation.
# history = model.fit_generator(gen(specs, labels, typeUsed, step_len = 8), steps_per_epoch=len(specs) //8, 
#                               epochs = 50, validation_data = (x_val, cat_y_val), callbacks=[checkpoint_loss, checkpoint_acc, earlystop, TestCallback((x_test, cat_y_test))])

history = model.fit(x=specs, y=cat_labels, validation_data=(x_val, cat_y_val), epochs=config["epoch"], batch_size=config["batch_size"], callbacks=[checkpoint_loss, earlystop, TestCallback((x_test, cat_y_test))])

In [ ]:
output_dir = "training_logs/"
output_file = output_dir + prefix
print(output_file)

In [ ]:
# Plot training history
hist_dic = history.history
fig, axs = plt.subplots(2, 1, figsize=(15,15))
axs[0].plot(hist_dic['loss'])
axs[0].plot(hist_dic['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])

axs[1].plot(hist_dic['accuracy'])
axs[1].plot(hist_dic['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

plt.savefig(output_file + ".jpg")

In [ ]:
df = pd.DataFrame.from_dict(hist_dic)
print("Saving file to:", output_file +".tsv")
df.to_csv(output_file + ".tsv", index=False)  

## Analyze

In [ ]:
# current model
results = model.evaluate( x= x_test, y=cat_y_test)
print("cur test loss, test acc:", results)
# print(model.predict(x_test))
results = model.evaluate( x= x_val, y=cat_y_val)
print("cur val loss, test acc:", results)

# load the model with best loss
model = None
keras.backend.clear_session()

model = load_model (filepath_loss)
# evaluation
results = model.evaluate( x= x_test, y=cat_y_test)
print("best loss test loss, test acc:", results)
results = model.evaluate( x= x_val, y=cat_y_val)
print("best loss val loss, test acc:", results)

### Details

In [ ]:
model = None
keras.backend.clear_session()
model = load_model (filepath_loss)
print("Using model:", filepath_loss)
y_scores = model.predict(x_test)
y_predict = np.argmax( y_scores, 1 )

In [ ]:
roc_auc_score(y_test, y_scores, multi_class="ovo")

In [ ]:
print(typeUsed)
print(confusion_matrix(y_test, y_predict))

In [ ]:
# differnce in y predict and y true
y_diff = [i for i in range(len(y_predict)) if y_predict[i] != y_test[i]]
print(y_diff)

In [ ]:
# Result for a specific sample.
index = 0
print("index", index)
print("True", y_test[index], typeUsed[int(y_test[index])])
print("Predict", y_predict[index], typeUsed[y_predict[index]])
print("Scores", y_scores[index])

plt.imshow(x_test[index][:,:,1], cmap='inferno', vmax=1)

In [ ]:
for i in range(len(typeUsed)):
  print(typeUsed[i])
  indices = np.setdiff1d(np.where(y_test==i), y_diff)
  print("Mean")
  mean_scores = np.mean(y_scores[indices], axis=0)
  print(mean_scores)
  print(mean_scores[i])

  print("Min")
  min_scores = np.min(y_scores[indices], axis=0)
  print(min_scores)
  print(min_scores[i])
  print()

### Test on all

In [ ]:
hdf5_file = 'path/to/data.hdf5'

f =  h5py.File(hdf5_file, "r+")
specs_h5 = np.array(f["specs"]).astype("float32")
labels_h5 = np.array(f["labels"])
sample_source_h5 = np.array(f["sample_source"])
f.close()

In [ ]:
specs_h5 = np.array(normalize(specs_h5))

y_scores = model.predict(specs_h5)
y_predict = np.argmax( y_scores, 1 )

In [ ]:
# Change label names to indices
typeUsed = [b'Anaxyrus-boreas',  b'Pseudacris-sierra',  b'Rana-boylii', b'Rana-catesbeiana',  b'Rana-draytonii',  b'noise-or-background']
label2index = {typeUsed[i]:i for i in range(len(typeUsed))}
y_test_all = np.array([label2index[x] for x in labels_h5])
y_test_all[:10]

In [ ]:
roc_auc_score(y_test_all, y_scores, multi_class="ovo")

In [ ]:
print(typeUsed)
print(confusion_matrix(y_test_all, y_predict))

In [ ]:
# differnce in y predict and y true
y_diff = [i for i in range(len(y_predict)) if y_predict[i] != y_test_all[i]]
print(y_diff, 1 - len(y_diff)/len(y_predict))

In [ ]:
# Result for a specific sample.
index = 0
print("index", index)
print("True", y_test_all[index], typeUsed[y_test_all[index]])
print("Predict", y_predict[index], typeUsed[y_predict[index]])
print("Scores", y_scores[index])
print("Source", sample_source_h5[index])
plt.imshow(specs_h5[index][:,:,1], cmap='inferno')

In [ ]:
for i in range(len(typeUsed)):
  print(typeUsed[i])
  indices = np.setdiff1d(np.where(y_test_all==i), y_diff)
  print(len(indices))
  print("Mean")
  mean_scores = np.mean(y_scores[indices], axis=0)
  print(mean_scores)
  print(mean_scores[i])

  print("Min")
  min_scores = np.min(y_scores[indices], axis=0)
  print(min_scores)
  print(min_scores[i])
  print()